In [16]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from matplotlib import dates
import plotly.graph_objs as go
import plotly.offline as pyoff

#import Keras
from tensorflow.keras.optimizers import Adam 
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from keras.layers import LSTM
from sklearn.model_selection import KFold, cross_val_score, train_test_split
import statsmodels.formula.api as smf

In [17]:
df = pd.read_excel('itService.xlsx')

In [18]:
# 예측데이터 추가
df= df.append({'YYMM' : '202201' , 'CNT' : 0} , ignore_index=True)
df

,YYMM,YY,MM,CNT
0,201305,2013.0,5.0,14
1,201306,2013.0,6.0,70
2,201307,2013.0,7.0,25
3,201308,2013.0,8.0,15
4,201309,2013.0,9.0,13
...,...,...,...,...
100,202109,2021.0,9.0,107
101,202110,2021.0,10.0,137
102,202111,2021.0,11.0,143
103,202112,2021.0,12.0,144


In [19]:
df['YYMM'] = df['YYMM'].astype(str)
df['YYMM'] = df['YYMM'] + "01"
df['YYMM'] = pd.to_datetime(df['YYMM'])
df

,YYMM,YY,MM,CNT
0,2013-05-01,2013.0,5.0,14
1,2013-06-01,2013.0,6.0,70
2,2013-07-01,2013.0,7.0,25
3,2013-08-01,2013.0,8.0,15
4,2013-09-01,2013.0,9.0,13
...,...,...,...,...
100,2021-09-01,2021.0,9.0,107
101,2021-10-01,2021.0,10.0,137
102,2021-11-01,2021.0,11.0,143
103,2021-12-01,2021.0,12.0,144


In [20]:
df.drop('YY', axis=1, inplace=True)
df.drop('MM', axis=1, inplace=True)
df

,YYMM,CNT
0,2013-05-01,14
1,2013-06-01,70
2,2013-07-01,25
3,2013-08-01,15
4,2013-09-01,13
...,...,...
100,2021-09-01,107
101,2021-10-01,137
102,2021-11-01,143
103,2021-12-01,144


In [21]:
df_diff = df.copy()
#add previous sales to the next row
df_diff['prev_CNT'] = df_diff['CNT'].shift(1)
#drop the null values and calculate the difference
df_diff = df_diff.dropna()
df_diff['diff'] = (df_diff['CNT'] - df_diff['prev_CNT'])
df_diff

,YYMM,CNT,prev_CNT,diff
1,2013-06-01,70,14.0,56.0
2,2013-07-01,25,70.0,-45.0
3,2013-08-01,15,25.0,-10.0
4,2013-09-01,13,15.0,-2.0
5,2013-10-01,19,13.0,6.0
...,...,...,...,...
100,2021-09-01,107,127.0,-20.0
101,2021-10-01,137,107.0,30.0
102,2021-11-01,143,137.0,6.0
103,2021-12-01,144,143.0,1.0


In [22]:
df_diff = df_diff[df['YYMM'].between('2021-01-01', '2022-01-01')]
df_diff

C:\Users\admin\AppData\Local\Temp/ipykernel_22120/2068859339.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



,YYMM,CNT,prev_CNT,diff
92,2021-01-01,72,13.0,59.0
93,2021-02-01,325,72.0,253.0
94,2021-03-01,362,325.0,37.0
95,2021-04-01,264,362.0,-98.0
96,2021-05-01,163,264.0,-101.0
97,2021-06-01,200,163.0,37.0
98,2021-07-01,180,200.0,-20.0
99,2021-08-01,127,180.0,-53.0
100,2021-09-01,107,127.0,-20.0
101,2021-10-01,137,107.0,30.0


In [23]:
#create dataframe for transformation from time series to supervised
df_supervised = df_diff.drop(['prev_CNT'],axis=1)
#adding lags
for inc in range(1,6):
    field_name = 'lag_' + str(inc)
    df_supervised[field_name] = df_supervised['diff'].shift(inc)
#drop null values
df_supervised = df_supervised.dropna().reset_index(drop=True)

In [24]:
df_supervised

,YYMM,CNT,diff,lag_1,lag_2,lag_3,lag_4,lag_5
0,2021-06-01,200,37.0,-101.0,-98.0,37.0,253.0,59.0
1,2021-07-01,180,-20.0,37.0,-101.0,-98.0,37.0,253.0
2,2021-08-01,127,-53.0,-20.0,37.0,-101.0,-98.0,37.0
3,2021-09-01,107,-20.0,-53.0,-20.0,37.0,-101.0,-98.0
4,2021-10-01,137,30.0,-20.0,-53.0,-20.0,37.0,-101.0
5,2021-11-01,143,6.0,30.0,-20.0,-53.0,-20.0,37.0
6,2021-12-01,144,1.0,6.0,30.0,-20.0,-53.0,-20.0
7,2022-01-01,0,-144.0,1.0,6.0,30.0,-20.0,-53.0


In [25]:
# Define the regression formula
model = smf.ols(formula='diff ~ lag_1 + lag_2 + lag_3 + lag_4 + lag_5', data=df_supervised)
# Fit the regression
model_fit = model.fit()
# Extract the adjusted r-squared
regression_adj_rsq = model_fit.rsquared_adj
print(regression_adj_rsq)

-0.863980973945472


In [26]:
#import MinMaxScaler and create a new dataframe for LSTM model
from sklearn.preprocessing import MinMaxScaler
df_model = df_supervised.drop(['CNT','YYMM'],axis=1)
#split train and test set
train_set, test_set = df_model[0:-1].values, df_model[-1:].values

In [27]:
train_set

array([[  37., -101.,  -98.,   37.,  253.,   59.],
       [ -20.,   37., -101.,  -98.,   37.,  253.],
       [ -53.,  -20.,   37., -101.,  -98.,   37.],
       [ -20.,  -53.,  -20.,   37., -101.,  -98.],
       [  30.,  -20.,  -53.,  -20.,   37., -101.],
       [   6.,   30.,  -20.,  -53.,  -20.,   37.],
       [   1.,    6.,   30.,  -20.,  -53.,  -20.]])

In [28]:
test_set

array([[-144.,    1.,    6.,   30.,  -20.,  -53.]])

In [29]:
#apply Min Max Scaler
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler = scaler.fit(train_set)
# reshape training set
train_set = train_set.reshape(train_set.shape[0], train_set.shape[1])
train_set_scaled = scaler.transform(train_set)
# reshape test set
test_set = test_set.reshape(test_set.shape[0], test_set.shape[1])
test_set_scaled = scaler.transform(test_set)

In [30]:
X_train, y_train = train_set_scaled[:, 1:], train_set_scaled[:, 0:1]
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test, y_test = test_set_scaled[:, 1:], test_set_scaled[:, 0:1]
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

In [31]:
model = Sequential()
model.add(LSTM(32, batch_input_shape=(1, X_train.shape[1], X_train.shape[2]), stateful=True))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, y_train, epochs=500, batch_size=1, verbose=1, shuffle=False)

Epoch 1/500
7/7 [==============================] - 1s 1ms/step - loss: 0.4806
Epoch 2/500
7/7 [==============================] - 0s 1ms/step - loss: 0.4696
Epoch 3/500
7/7 [==============================] - 0s 1ms/step - loss: 0.4389
Epoch 4/500
7/7 [==============================] - 0s 1ms/step - loss: 0.4116
Epoch 5/500
7/7 [==============================] - 0s 1ms/step - loss: 0.3901
Epoch 6/500
7/7 [==============================] - 0s 1ms/step - loss: 0.3727
Epoch 7/500
7/7 [==============================] - 0s 1ms/step - loss: 0.3579
Epoch 8/500
7/7 [==============================] - 0s 1ms/step - loss: 0.3447
Epoch 9/500
7/7 [==============================] - 0s 1ms/step - loss: 0.3323
Epoch 10/500
7/7 [==============================] - 0s 1ms/step - loss: 0.3203
Epoch 11/500
7/7 [==============================] - 0s 1ms/step - loss: 0.3082
Epoch 12/500
7/7 [==============================] - 0s 1ms/step - loss: 0.2960
Epoch 13/500
7/7 [==============================] - 0s 1ms/st

In [33]:
model.evaluate(X_test, y_test)

1/1 [==============================] - 0s 308ms/step - loss: 13.7274


13.7273530960083

In [35]:
y_pred = model.predict(X_test,batch_size=1)
y_pred

array([[0.89182854]], dtype=float32)

In [37]:
#reshape y_pred
y_pred = y_pred.reshape(y_pred.shape[0], 1, y_pred.shape[1])
#rebuild test set for inverse transform
pred_test_set = []
for index in range(0,len(y_pred)):
    print(np.concatenate([y_pred[index],X_test[index]],axis=1))
    pred_test_set.append(np.concatenate([y_pred[index],X_test[index]],axis=1))
#reshape pred_test_set
pred_test_set = np.array(pred_test_set)
pred_test_set = pred_test_set.reshape(pred_test_set.shape[0], pred_test_set.shape[2])
#inverse transform
pred_test_set_inverted = scaler.inverse_transform(pred_test_set)
pred_test_set_inverted


[[ 0.89182854  0.47826087  0.55072464  0.89855072 -0.54237288 -0.72881356]]


array([[ 32.13228416,   1.        ,   6.        ,  30.        ,
        -20.        , -53.        ]])

In [39]:
df_supervised

,YYMM,CNT,diff,lag_1,lag_2,lag_3,lag_4,lag_5
0,2021-06-01,200,37.0,-101.0,-98.0,37.0,253.0,59.0
1,2021-07-01,180,-20.0,37.0,-101.0,-98.0,37.0,253.0
2,2021-08-01,127,-53.0,-20.0,37.0,-101.0,-98.0,37.0
3,2021-09-01,107,-20.0,-53.0,-20.0,37.0,-101.0,-98.0
4,2021-10-01,137,30.0,-20.0,-53.0,-20.0,37.0,-101.0
5,2021-11-01,143,6.0,30.0,-20.0,-53.0,-20.0,37.0
6,2021-12-01,144,1.0,6.0,30.0,-20.0,-53.0,-20.0
7,2022-01-01,0,-144.0,1.0,6.0,30.0,-20.0,-53.0


In [42]:
# diff 값
pred_test_set_inverted[:,0:1][0][0]

32.13228416442871

In [44]:
df_supervised.iloc[7,2] = pred_test_set_inverted[:,0:1][0][0]
df_supervised

,YYMM,CNT,diff,lag_1,lag_2,lag_3,lag_4,lag_5
0,2021-06-01,200,37.000000,-101.0,-98.0,37.0,253.0,59.0
1,2021-07-01,180,-20.000000,37.0,-101.0,-98.0,37.0,253.0
2,2021-08-01,127,-53.000000,-20.0,37.0,-101.0,-98.0,37.0
3,2021-09-01,107,-20.000000,-53.0,-20.0,37.0,-101.0,-98.0
4,2021-10-01,137,30.000000,-20.0,-53.0,-20.0,37.0,-101.0
5,2021-11-01,143,6.000000,30.0,-20.0,-53.0,-20.0,37.0
6,2021-12-01,144,1.000000,6.0,30.0,-20.0,-53.0,-20.0
7,2022-01-01,0,32.132284,1.0,6.0,30.0,-20.0,-53.0


In [45]:
df_supervised.shape

(8, 8)

In [47]:
df_supervised.iloc[7,1] = df_supervised.iloc[6,1] - df_supervised.iloc[7,2]
df_supervised

,YYMM,CNT,diff,lag_1,lag_2,lag_3,lag_4,lag_5
0,2021-06-01,200.000000,37.000000,-101.0,-98.0,37.0,253.0,59.0
1,2021-07-01,180.000000,-20.000000,37.0,-101.0,-98.0,37.0,253.0
2,2021-08-01,127.000000,-53.000000,-20.0,37.0,-101.0,-98.0,37.0
3,2021-09-01,107.000000,-20.000000,-53.0,-20.0,37.0,-101.0,-98.0
4,2021-10-01,137.000000,30.000000,-20.0,-53.0,-20.0,37.0,-101.0
5,2021-11-01,143.000000,6.000000,30.0,-20.0,-53.0,-20.0,37.0
6,2021-12-01,144.000000,1.000000,6.0,30.0,-20.0,-53.0,-20.0
7,2022-01-01,111.867716,32.132284,1.0,6.0,30.0,-20.0,-53.0
